In [58]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import scale


In [59]:
df = pd.read_csv("boston.csv",header=0)
print(df)

         CRIM   ZN   INDUS   CHAS    NOX     RM    AGE     DIS  RAD  TAX  \
0     0.00632  18.0    2.31     0  0.538  6.575   65.2  4.0900    1  296   
1     0.02731   0.0    7.07     0  0.469  6.421   78.9  4.9671    2  242   
2     0.02729   0.0    7.07     0  0.469  7.185   61.1  4.9671    2  242   
3     0.03237   0.0    2.18     0  0.458  6.998   45.8  6.0622    3  222   
4     0.06905   0.0    2.18     0  0.458  7.147   54.2  6.0622    3  222   
5     0.02985   0.0    2.18     0  0.458  6.430   58.7  6.0622    3  222   
6     0.08829  12.5    7.87     0  0.524  6.012   66.6  5.5605    5  311   
7     0.14455  12.5    7.87     0  0.524  6.172   96.1  5.9505    5  311   
8     0.21124  12.5    7.87     0  0.524  5.631  100.0  6.0821    5  311   
9     0.17004  12.5    7.87     0  0.524  6.004   85.9  6.5921    5  311   
10    0.22489  12.5    7.87     0  0.524  6.377   94.3  6.3467    5  311   
11    0.11747  12.5    7.87     0  0.524  6.009   82.9  6.2267    5  311   
12    0.0937

In [60]:
df = df
print(df)
df =np.array(df)
print(df)
x_data = df[:,:12]
y_data = df[:,12]
print(x_data,'\n shape=',x_data.shape)
print(y_data,'\n shape=',y_data.shape)

         CRIM   ZN   INDUS   CHAS    NOX     RM    AGE     DIS  RAD  TAX  \
0     0.00632  18.0    2.31     0  0.538  6.575   65.2  4.0900    1  296   
1     0.02731   0.0    7.07     0  0.469  6.421   78.9  4.9671    2  242   
2     0.02729   0.0    7.07     0  0.469  7.185   61.1  4.9671    2  242   
3     0.03237   0.0    2.18     0  0.458  6.998   45.8  6.0622    3  222   
4     0.06905   0.0    2.18     0  0.458  7.147   54.2  6.0622    3  222   
5     0.02985   0.0    2.18     0  0.458  6.430   58.7  6.0622    3  222   
6     0.08829  12.5    7.87     0  0.524  6.012   66.6  5.5605    5  311   
7     0.14455  12.5    7.87     0  0.524  6.172   96.1  5.9505    5  311   
8     0.21124  12.5    7.87     0  0.524  5.631  100.0  6.0821    5  311   
9     0.17004  12.5    7.87     0  0.524  6.004   85.9  6.5921    5  311   
10    0.22489  12.5    7.87     0  0.524  6.377   94.3  6.3467    5  311   
11    0.11747  12.5    7.87     0  0.524  6.009   82.9  6.2267    5  311   
12    0.0937

In [61]:
train_num = 300
valid_num = 100
test_num = len(x_data)- train_num -valid_num

x_train = x_data[:train_num]
y_train = y_data[:train_num]

x_valid = x_data[train_num:train_num+valid_num]
y_valid = y_data[train_num:train_num+valid_num]

x_test = x_data[train_num+valid_num:train_num+valid_num+test_num]
y_test = y_data[train_num+valid_num:train_num+valid_num+test_num]

x_train = tf.cast(scale(x_train), dtype=tf.float32)
x_valid = tf.cast(scale(x_valid), dtype=tf.float32)
x_test = tf.cast(scale(x_test), dtype=tf.float32)

In [62]:
def model(x, w, b):
    return tf.matmul(x,w)+ b

W = tf.Variable(tf.random.normal([12,1],mean=0.0, stddev = 1.0, dtype=tf.float32))
B = tf.Variable(tf.zeros(1), dtype = tf.float32)

print(W)
print(B)

<tf.Variable 'Variable:0' shape=(12, 1) dtype=float32, numpy=
array([[ 0.9763091 ],
       [-0.3685155 ],
       [ 0.49096242],
       [ 0.5959448 ],
       [-0.06455412],
       [-0.6748196 ],
       [ 0.08178204],
       [-1.4440577 ],
       [-0.69310975],
       [-1.8991889 ],
       [-1.3837553 ],
       [ 1.5373732 ]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>


In [63]:
training_epochs = 500
learning_rate = 0.0001
batch_size = 10
def loss(x,y,w,b):
    err = model(x,w,b)-y
    squared_err = tf.square(err)
    return tf.reduce_mean(squared_err)
def grad(x,y,w,b):
    with tf.GradientTape() as tape:
        loss_ = loss(x,y,w,b)
    return tape.gradient(loss_, [w,b])
optimizer = tf.keras.optimizers.SGD(learning_rate)

In [64]:
loss_list_train = []
loss_list_valid = []
total_step = int (train_num/batch_size)

for epoch in range(training_epochs):
    for step in range(total_step):
        xs = x_train[step*batch_size:(step+1)*batch_size,:]
        ys = y_train[step*batch_size:(step+1)*batch_size]
        
        grads = grad(xs,ys,W,B)
        optimizer.apply_gradients(zip(grads,[W,B]))
        
    loss_train = loss(x_train, y_train,W ,B).numpy()
    loss_valid = loss(x_valid, y_valid,W ,B).numpy()
    loss_list_train.append(loss_train)
    loss_list_valid.append(loss_valid)
    print("epoch={:3d} ,train_loss={:.4f},valid_loss={:.4f}".format(epoch+1,loss_train,loss_valid))

epoch=  1 ,train_loss=735.8864,valid_loss=519.0682
epoch=  2 ,train_loss=727.3947,valid_loss=512.4758
epoch=  3 ,train_loss=719.0653,valid_loss=506.0312
epoch=  4 ,train_loss=710.8928,valid_loss=499.7284
epoch=  5 ,train_loss=702.8717,valid_loss=493.5618
epoch=  6 ,train_loss=694.9971,valid_loss=487.5259
epoch=  7 ,train_loss=687.2638,valid_loss=481.6158
epoch=  8 ,train_loss=679.6675,valid_loss=475.8263
epoch=  9 ,train_loss=672.2037,valid_loss=470.1533
epoch= 10 ,train_loss=664.8683,valid_loss=464.5920
epoch= 11 ,train_loss=657.6573,valid_loss=459.1386
epoch= 12 ,train_loss=650.5671,valid_loss=453.7892
epoch= 13 ,train_loss=643.5938,valid_loss=448.5400
epoch= 14 ,train_loss=636.7341,valid_loss=443.3877
epoch= 15 ,train_loss=629.9849,valid_loss=438.3289
epoch= 16 ,train_loss=623.3428,valid_loss=433.3605
epoch= 17 ,train_loss=616.8050,valid_loss=428.4797
epoch= 18 ,train_loss=610.3687,valid_loss=423.6836
epoch= 19 ,train_loss=604.0311,valid_loss=418.9695
epoch= 20 ,train_loss=597.7897,

epoch=163 ,train_loss=190.7837,valid_loss=134.6965
epoch=164 ,train_loss=189.7229,valid_loss=134.1351
epoch=165 ,train_loss=188.6747,valid_loss=133.5833
epoch=166 ,train_loss=187.6390,valid_loss=133.0408
epoch=167 ,train_loss=186.6158,valid_loss=132.5077
epoch=168 ,train_loss=185.6047,valid_loss=131.9838
epoch=169 ,train_loss=184.6057,valid_loss=131.4690
epoch=170 ,train_loss=183.6185,valid_loss=130.9631
epoch=171 ,train_loss=182.6431,valid_loss=130.4661
epoch=172 ,train_loss=181.6795,valid_loss=129.9778
epoch=173 ,train_loss=180.7272,valid_loss=129.4981
epoch=174 ,train_loss=179.7864,valid_loss=129.0269
epoch=175 ,train_loss=178.8568,valid_loss=128.5640
epoch=176 ,train_loss=177.9383,valid_loss=128.1095
epoch=177 ,train_loss=177.0307,valid_loss=127.6630
epoch=178 ,train_loss=176.1340,valid_loss=127.2246
epoch=179 ,train_loss=175.2480,valid_loss=126.7942
epoch=180 ,train_loss=174.3725,valid_loss=126.3716
epoch=181 ,train_loss=173.5075,valid_loss=125.9566
epoch=182 ,train_loss=172.6528,

epoch=329 ,train_loss=114.1612,valid_loss=110.9002
epoch=330 ,train_loss=114.0181,valid_loss=110.9453
epoch=331 ,train_loss=113.8768,valid_loss=110.9910
epoch=332 ,train_loss=113.7371,valid_loss=111.0374
epoch=333 ,train_loss=113.5992,valid_loss=111.0843
epoch=334 ,train_loss=113.4629,valid_loss=111.1319
epoch=335 ,train_loss=113.3283,valid_loss=111.1800
epoch=336 ,train_loss=113.1952,valid_loss=111.2288
epoch=337 ,train_loss=113.0639,valid_loss=111.2780
epoch=338 ,train_loss=112.9341,valid_loss=111.3278
epoch=339 ,train_loss=112.8058,valid_loss=111.3782
epoch=340 ,train_loss=112.6791,valid_loss=111.4290
epoch=341 ,train_loss=112.5540,valid_loss=111.4804
epoch=342 ,train_loss=112.4303,valid_loss=111.5323
epoch=343 ,train_loss=112.3081,valid_loss=111.5847
epoch=344 ,train_loss=112.1874,valid_loss=111.6375
epoch=345 ,train_loss=112.0682,valid_loss=111.6908
epoch=346 ,train_loss=111.9503,valid_loss=111.7446
epoch=347 ,train_loss=111.8340,valid_loss=111.7987
epoch=348 ,train_loss=111.7190,

epoch=494 ,train_loss=103.9412,valid_loss=120.7955
epoch=495 ,train_loss=103.9224,valid_loss=120.8500
epoch=496 ,train_loss=103.9039,valid_loss=120.9042
epoch=497 ,train_loss=103.8856,valid_loss=120.9584
epoch=498 ,train_loss=103.8675,valid_loss=121.0123
epoch=499 ,train_loss=103.8497,valid_loss=121.0661
epoch=500 ,train_loss=103.8321,valid_loss=121.1196


In [67]:
print("Test_loss:{:.4f}".format(loss(x_test, y_test, W, B).numpy()))
#print("W:{:.4f}".format(W.numpy()))
#print("B:{:.4f}".format(B))
print(W)
print(B)

Test_loss:113.5717
<tf.Variable 'Variable:0' shape=(12, 1) dtype=float32, numpy=
array([[ 0.29776895],
       [ 0.3685904 ],
       [-0.66927135],
       [ 0.67803794],
       [-1.8162059 ],
       [ 1.4693649 ],
       [-0.22928289],
       [-1.2676358 ],
       [ 0.58518386],
       [-0.76447606],
       [-2.5849106 ],
       [-0.71795756]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([24.220371], dtype=float32)>
